In [3]:
import numpy as np
import pandas as pd

In [4]:
##here we will program an heston model
#stochastic volatility model
###general formula:
### dSt = (r-q)St*dt + sqrt(V)*dZs
### dV = a(Vl-V)dt + xi*V**(alpha) * dZv

def monte_carlo_heston(S0: float, a: float, alpha: float, xi: float, V0: float, VL: float, r: float, q: float, T: float, NoOfSteps: int, NoOfPaths: int) -> pd.DataFrame:
    """
    :param S0: Initial stock price
    :param a: mean revert rate
    :param alpha: exponent of Vol
    :param xi: volatility of volatility (kurtosis)
    :param V0: initial volatility
    :param VL: Long term Variance
    :param r: riskless rate
    :param rho: correlation coefficient between returns and vol
    :param q: dividend yield
    :param T: Time to maturity
    :param NoOfSteps: Number of steps on each random walk
    :param NoOfPaths: Number of random walks
    :return: Stochastic Volatility paths and Stock paths
    """
    dt = T/NoOfSteps
    drift = r - q
    cov = np.array([[1, rho],
                    [rho, 1]])
    # sampling correlated brownian motions under risk-neutral measure
    correlated_BM = np.random.multivariate_normal([0, 0], cov, (NoOfSteps, NoOfPaths))
    samples_S = correlated_BM[:, :, 0]
    paths_S = np.ones(shape=[NoOfSteps, NoOfPaths])
    samples_V = correlated_BM[:, :, 1]
    paths_V = np.ones(shape=[NoOfSteps, NoOfPaths])
    paths_V[0, :] = V0
    paths_S[0, :] = S0
    for i in range(1, NoOfSteps):
        for n in range(NoOfPaths):
            paths_V[i, n] = np.maximum(paths_V[i - 1, n] + a*(VL - paths_V[i-1, n])*dt + xi*(paths_V[i-1, n]**alpha) * samples_V[i, n]*np.sqrt(dt), 0)
            paths_S[i, n] = paths_S[i-1, n]*np.exp(drift*dt + np.sqrt(paths_V[i-1, n]) * samples_S[i, n] * np.sqrt(dt))
    vol = [f"vol_{i}" for i in range(NoOfPaths)]
    stock = [f"stock process_{i}" for i in range(NoOfPaths)]
    vol = pd.DataFrame(data=paths_V, columns = vol)
    S = pd.DataFrame(data=paths_S, columns = stock)
    return vol, S

# distribution moments

if __name__ == '__main__':
    """
    parameters : 
    St: stock price at time t
    r: risk free rate
    q: dividend yield
    dt: time step
    V: vol
    rho: correlation between vol and returns
    dzt and dwt: uncorrelated brownian motions
    alpha : constant
    xi: constant (Long term variance)
    a: mean revert rate
    """
    #parameters
    # V0 = 0.1
    # VL = 0.1
    # S0 = 100
    # a = 3
    # T = 1
    # xi = 0.5
    # alpha = 0.5
    # NoOfSteps = 252
    # NoOfPaths = 1000
    # r = 0.04
    # rho = 0 
    # q = 0.01
    # Vol, S = monte_carlo_heston(S0, a, alpha, xi, V0, VL, r, q, T, NoOfSteps, NoOfPaths)


In [5]:
def mean(n:int, returns: pd.DataFrame) -> pd.DataFrame:
    """
    :param n: n periods mean
    :param returns: time series returns
    :return: skew of the n returns
    """
    return returns.iloc[:n].mean()

def standard_deviation(n:int, returns: pd.DataFrame) -> pd.DataFrame:
    """
    :param n: n periods std
    :param returns: time series returns
    :return: std of the n returns
    """
    return returns.iloc[:n].std()

def skewness(n: int, returns:pd.DataFrame) -> pd.DataFrame:
    """
    :param n: n periods skew
    :param returns: time series returns
    :return: skew of the n returns
    """
    return returns.iloc[:n].skew()

def kurtosis(n:int, returns:pd.DataFrame) -> pd.DataFrame:
    """
    :param n: n periods kurtosis
    :param returns: time series returns
    :return: kurtosis of the n returns
    """
    return returns.iloc[:n].kurtosis()